# Scraping coinmarketcap for ICO prices
Leon Yin | Github: [yinleon](#TODO) | Twitter: [@leonyin](#TODO) | Updated: 2017-11-12

This is notebook that explains how to make a scraper that collects a [table of ICO stats](https://coinmarketcap.com/all/views/all/) from the site coinmarketcap.com. 

While we're at it we create a few helpful metadata columns, makes numerical values machine-readible, and perform some simple data analysis. Because this is a Jupyter Notebook you can run it on your own machine :)


View this on Github [here](#TODO) or NBViewer [here](#TODO).<br>
The scraper is available as Python script [here](#TODO).<br>
The hourly data is avilable open source on Amazon s3 [here](#TODO)

If you like this project please help support it by contributing time to make it better, or donating to help [pay for hosting](#TODO).

### Table of Contents
1. [Scraping Data with Requests and Beautiful Soup](#scrape)
2. [Cleaning Data with Pandas](#clean)
3. [Analysis with Pandas](#analysis)
4. [Next Steps](#next-steps)

In [1]:
import os
import re
import datetime
import requests

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
root_dir = '../'
table_url = 'https://coinmarketcap.com/all/views/all/'
table_id = 'currencies-all'
today = datetime.datetime.now()

## Scraping a Website with Requests and Beautiful Soup <a id='scrape'></a>
Let's visit the coinmarketcap website programatically using the requests package...

In [3]:
r = requests.get(table_url)

Among other things, `r` contains the html content of the page we visited.

In [4]:
r.content[:300]

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> \n<html class="no-js" lang="en"> <!--<![endif]-->'

BeautifulSoup is the defacto package (still?) for parsing HTML content.<br>
We can send the html from `r` into a parsable object.

In [5]:
soup = BeautifulSoup(r.content, 'lxml')

This new object (`soup`) comes in handy because we can isolate sections of the HTML page using `soup.find()`.<br>
The seciton we are after is the element that contains the ICO data.<br>
We can use the `inspect element` feature from Chrome to identify the ID (in this case currencies-all) of this table.

<img src='../media/find_element.png'></img>

I stored the ID for this table as a variable `table_id`.

In [6]:
table_id

'currencies-all'

Let's isolate the table element using `table_id`, and read it into a Pandas dataframe.

In [7]:
html_tbl = str(soup.find('table',{'id': table_id}))
df = pd.read_html(html_tbl, index_col=0)[0]

We can get a peak at the top 5 ICOs.

In [8]:
df.head()

,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d
#,,,,,,,,,
1,BTC Bitcoin,BTC,"$110,429,726,504",$6621.43,16677625,"$6,806,100,000",-1.29%,6.52%,-9.00%
2,ETH Ethereum,ETH,"$30,150,574,032",$315.05,95701829,"$1,381,640,000",-0.09%,3.29%,4.44%
3,BCH Bitcoin Cash,BCH,"$20,821,268,310",$1239.34,16800288,"$4,514,090,000",12.68%,-20.95%,96.96%
4,XRP Ripple,XRP,"$7,769,229,787",$0.201633,"38,531,538,922 *","$162,209,000",-0.78%,1.55%,-2.21%
5,LTC Litecoin,LTC,"$3,267,915,303",$60.73,53811357,"$299,954,000",-1.04%,3.01%,9.36%


Now that we have everything in Pandas, we can do some extra janitorial work, and analysis.<br>
If you've never used Pandas, you're in for a treat!

## Cleaning Data with Pandas <a id='clean'></a>
The data we have is straight from the HTML table--<br>
it is human-readible, but not machine readible.

Let's use Pandas to clean up the the data (stored in a DataFrame `df`)...<br>
As a first step, we can add a timestamp for context.

In [9]:
df['scrape_timestamp'] = today

### Renaming Columns
Let's make the columns more descriptive, by including the unit in each column (USD).<br>
We can do this by replacing column names using a key-value store (a dictionary)

In [10]:
col_name_w_currency = {
    'Market Cap' : 'market_cap_usd',
    'Price' : 'price_usd',
    'Volume (24h)': 'volume_24h_usd',
}

and a function that operates on each column name.

In [11]:
def clean_up_col(col):
    '''
    Adds currency unit to relevant column names,
    replaces spaces for underscores, 
    replaces % symbols for "percent_change",
    and returns the updated column in lower case.
    '''
    col = col_name_w_currency.get(col, col)
    col = col.replace(' ', '_')
    col = col.replace('%', 'percent_change')
    return col.lower()

Now let's iterate through each column name in `df`, and apply `clean_up_col` to each.<br>
For reference: `[x for x in some_iterator]` is called a list comprehension, which is a slight modification to a for loop.

In [12]:
df.columns = [clean_up_col(c) for c in df.columns]

In [13]:
df.dtypes

name                          object
symbol                        object
market_cap_usd                object
price_usd                     object
circulating_supply            object
volume_24h_usd                object
percent_change_1h             object
percent_change_24h            object
percent_change_7d             object
scrape_timestamp      datetime64[ns]
dtype: object

In [14]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp
#,,,,,,,,,,
1,BTC Bitcoin,BTC,"$110,429,726,504",$6621.43,16677625,"$6,806,100,000",-1.29%,6.52%,-9.00%,2017-11-13 09:45:14.163318
2,ETH Ethereum,ETH,"$30,150,574,032",$315.05,95701829,"$1,381,640,000",-0.09%,3.29%,4.44%,2017-11-13 09:45:14.163318
3,BCH Bitcoin Cash,BCH,"$20,821,268,310",$1239.34,16800288,"$4,514,090,000",12.68%,-20.95%,96.96%,2017-11-13 09:45:14.163318
4,XRP Ripple,XRP,"$7,769,229,787",$0.201633,"38,531,538,922 *","$162,209,000",-0.78%,1.55%,-2.21%,2017-11-13 09:45:14.163318


Notice that some columns have an asterix (used to denote that the currency are not minable).<br>
We can convert this feature into a new column by leveraging Pandas DataFrames' `apply` function -- <br>
which applies any function (anonymous or declared) across either columns (`axis`=0) or rows (`axis`=1).

In [15]:
def is_minable(row):
    '''
    Check if `circulating_supply` contains an asterix.
    This function operates on each row of the dataframe.
    If the ICO is not minable, we'll find an asterix and return 0.
    
    Note:
    That when we apply a function across a row,
    the entire row is treated as a key-value pair.
    '''
    circulating_supply = row['circulating_supply']
    
    if '*' in circulating_supply:
        return 0
    
    else:
        return 1

In [16]:
df['is_minable'] = df.apply(is_minable, axis=1)

In [17]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
#,,,,,,,,,,,
1,BTC Bitcoin,BTC,"$110,429,726,504",$6621.43,16677625,"$6,806,100,000",-1.29%,6.52%,-9.00%,2017-11-13 09:45:14.163318,1
2,ETH Ethereum,ETH,"$30,150,574,032",$315.05,95701829,"$1,381,640,000",-0.09%,3.29%,4.44%,2017-11-13 09:45:14.163318,1
3,BCH Bitcoin Cash,BCH,"$20,821,268,310",$1239.34,16800288,"$4,514,090,000",12.68%,-20.95%,96.96%,2017-11-13 09:45:14.163318,1
4,XRP Ripple,XRP,"$7,769,229,787",$0.201633,"38,531,538,922 *","$162,209,000",-0.78%,1.55%,-2.21%,2017-11-13 09:45:14.163318,0


### This table is now more human readible, but problematic for machines
Why? Because there are dollar signs, commas, asterix, and percent signs in numeric values.<br>
This causes most computers (and Pandas) to view numeric values as strings!

In [18]:
df.dtypes

name                          object
symbol                        object
market_cap_usd                object
price_usd                     object
circulating_supply            object
volume_24h_usd                object
percent_change_1h             object
percent_change_24h            object
percent_change_7d             object
scrape_timestamp      datetime64[ns]
is_minable                     int64
dtype: object

We can remove these symbols using regular expressions.<br>
Below is a dictionary of regular expressions we can use to weed out symbols

In [19]:
replace_symbols = {
    r'  [*]' : '',    # two spaces and any number of asterix
    r'[\$,%*]' : '',  # money signs, commas, percent signs, asterix
    r'[?]' : np.nan,  # question marks becomes a null value
    'Low Vol' : 0,    # low volume is simplified as zero...
}

Pandas `replace` operates on all all columns and all rows.<br>
The coolest aspect of this function is that 
1. it can take a dictionary as an input,
2. it can implement regular expressions, and
3. it can operate inplace

In [20]:
df.replace(replace_symbols, regex=True, inplace=True)

In [21]:
df.head(4)

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
#,,,,,,,,,,,
1,BTC Bitcoin,BTC,110429726504,6621.43,16677625,6806100000,-1.29,6.52,-9.00,2017-11-13 09:45:14.163318,1
2,ETH Ethereum,ETH,30150574032,315.05,95701829,1381640000,-0.09,3.29,4.44,2017-11-13 09:45:14.163318,1
3,BCH Bitcoin Cash,BCH,20821268310,1239.34,16800288,4514090000,12.68,-20.95,96.96,2017-11-13 09:45:14.163318,1
4,XRP Ripple,XRP,7769229787,0.201633,38531538922,162209000,-0.78,1.55,-2.21,2017-11-13 09:45:14.163318,0


This looks good to me! Let's write this clean dataset to a csv.<br>
A best practice in data engineering is to create a function to programmatically generate file paths and directories.

In [22]:
def create_filename(root_dir, today):
    '''
    This function creates the filename, 
    it also creates the directory for the file if the directory doesn't exist.
    '''
    f_template = '{year}/{month}/{day}/{hour}/market_cap_USD_{time}.csv.gz'
    f = f_template.format(year = today.year,
                          month= today.month,
                          day  = today.day,
                          hour = today.strftime('%H'),
                          time = today.strftime('%H:%M:%S'))
    
    f_out = os.path.join(root_dir, f)
    
    dir_out = '/'.join(f_out.split('/')[:-1])
    if not os.path.exists(dir_out):
        os.makedirs(dir_out, exist_ok=True)
    
    return f_out

In [23]:
file = create_filename(root_dir, today)
file

'../2017/11/13/09/market_cap_USD_09:45:14.csv.gz'

In [24]:
df.to_csv(file, index=None, compression='gzip')

## Let's do some analysis <a id='analysis'></a>
How's the data look?

In [25]:
df = pd.read_csv(file, compression='gzip')

In [26]:
df.head()

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
0,BTC Bitcoin,BTC,1.104297e+11,6621.430000,1.667762e+07,6.806100e+09,-1.29,6.52,-9.00,2017-11-13 09:45:14.163318,1
1,ETH Ethereum,ETH,3.015057e+10,315.050000,9.570183e+07,1.381640e+09,-0.09,3.29,4.44,2017-11-13 09:45:14.163318,1
2,BCH Bitcoin Cash,BCH,2.082127e+10,1239.340000,1.680029e+07,4.514090e+09,12.68,-20.95,96.96,2017-11-13 09:45:14.163318,1
3,XRP Ripple,XRP,7.769230e+09,0.201633,3.853154e+10,1.622090e+08,-0.78,1.55,-2.21,2017-11-13 09:45:14.163318,0
4,LTC Litecoin,LTC,3.267915e+09,60.730000,5.381136e+07,2.999540e+08,-1.04,3.01,9.36,2017-11-13 09:45:14.163318,1


Monetary and percentages are now floats!

In [27]:
df.dtypes

name                   object
symbol                 object
market_cap_usd        float64
price_usd             float64
circulating_supply    float64
volume_24h_usd        float64
percent_change_1h     float64
percent_change_24h    float64
percent_change_7d     float64
scrape_timestamp       object
is_minable              int64
dtype: object

We can get a big picture of what's going on:


In [28]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
market_cap_usd,989.0,2.062930e+08,3.709711e+09,4.000000e+00,9.178800e+04,9.139370e+05,9.453571e+06,1.104297e+11
price_usd,1277.0,2.601015e+02,5.829740e+03,1.600000e-08,2.265000e-03,2.939500e-02,3.032810e-01,1.836010e+05
circulating_supply,989.0,5.732600e+09,5.039963e+10,7.826400e-02,4.886565e+06,2.209930e+07,1.088351e+08,1.059921e+12
volume_24h_usd,1265.0,1.274583e+07,2.350331e+08,0.000000e+00,0.000000e+00,1.513000e+03,5.951600e+04,6.806100e+09
percent_change_1h,1188.0,-1.304167e+00,1.040415e+01,-3.716000e+01,-3.260000e+00,-3.120000e+00,-1.117500e+00,1.906400e+02
percent_change_24h,1191.0,1.171171e+01,7.933950e+01,-8.247000e+01,-3.500000e-02,4.630000e+00,1.193500e+01,2.366840e+03
percent_change_7d,1200.0,1.173899e+01,1.835343e+02,-9.897000e+01,-1.787500e+01,-5.860000e+00,1.217500e+01,5.584360e+03
is_minable,1278.0,4.749609e-01,4.995681e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00


There are aggregation functions we can use to calculate the market cap:

In [29]:
df['market_cap_usd'].sum()

204023780750.0

We can also find which ICOs have dropped by more than 60% since the past week:

In [30]:
df_losers = df[df['percent_change_7d'] <= -60]
df_losers.head()

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
197,MCAP MCAP,MCAP,13872826.0,0.362256,3.829564e+07,738547.0,-2.81,-9.19,-71.41,2017-11-13 09:45:14.163318,0
417,NYC NewYorkCoin,NYC,1824304.0,0.000014,1.290328e+11,3801.0,4.89,3.46,-65.62,2017-11-13 09:45:14.163318,1
481,ARC Arcade Token,ARC,697983.0,0.386318,1.806758e+06,652.0,0.21,-74.63,-79.77,2017-11-13 09:45:14.163318,0
482,NAUT NautilusCoin,NAUT,697546.0,0.043112,1.618000e+07,56847.0,-3.27,2.33,-62.21,2017-11-13 09:45:14.163318,0
501,GRE Greencoin,GRE,549279.0,0.000156,3.528417e+09,1825.0,-14.56,-68.43,-92.58,2017-11-13 09:45:14.163318,0


Since we don't care about EVERY ICO, we can filter the dataframe by relevant symbols

In [31]:
watchlist = [
    'LTC',
    'BTC',
    'NEO'
]

In [32]:
df_w = df[df['symbol'].isin(watchlist)]
df_w

,name,symbol,market_cap_usd,price_usd,circulating_supply,volume_24h_usd,percent_change_1h,percent_change_24h,percent_change_7d,scrape_timestamp,is_minable
0,BTC Bitcoin,BTC,1.104297e+11,6621.43,16677625.0,6.806100e+09,-1.29,6.52,-9.00,2017-11-13 09:45:14.163318,1
4,LTC Litecoin,LTC,3.267915e+09,60.73,53811357.0,2.999540e+08,-1.04,3.01,9.36,2017-11-13 09:45:14.163318,1
7,NEO NEO,NEO,1.812076e+09,27.88,65000000.0,4.689940e+07,-2.58,3.41,5.70,2017-11-13 09:45:14.163318,0


We can also calculate values in BTC

In [33]:
btc_price = df[df['symbol'] == 'BTC']['price_usd'].iloc[0]
btc_price

6621.4300000000003

In [34]:
df_w['price_btc'] = df_w['price_usd'] / btc_price

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
df_w[['name', 'symbol', 'price_usd', 'price_btc']].head()

,name,symbol,price_usd,price_btc
0,BTC Bitcoin,BTC,6621.43,1.000000
4,LTC Litecoin,LTC,60.73,0.009172
7,NEO NEO,NEO,27.88,0.004211


The data we just scraped is also [available in BTC](https://coinmarketcap.com/coins/views/all/#BTC), rather than USD.<br>
However, that table is rendered using Javascript, <br>
so it can't be scraped unless we use a client such as Selenium.

### One application is calculating your portfolio's worth

In [36]:
portfolio = [
    {'symbol' : 'BTC', 'quantity' : 2.045},
    {'symbol' : 'ETH', 'quantity' : 20.17},
    {'symbol' : 'DASH', 'quantity' : 1001},
]

In [59]:
def get_portfolio(df, portolio, cols=['name', 'symbol', 'price_usd', 'percent_change_24h', 
                                      'quantity', 'value_usd', 'value_btc']):
    '''
    Converts a dict `portfolio` into a pandas dataframe `df_p`,
    Inner join `df_p` to a dataframe of ICO prices (`df`).
    Calculates and printsvalue in USD and BTC.
    
    Retuns a dataframe with `cols` colums from the joined dataframe `df_`.
    '''
    df_p = pd.DataFrame(portfolio)
    btc_price = df[df['symbol'] == 'BTC']['price_usd'].iloc[0]
    
    df_ = df.merge(df_p, how='inner')
    df_['value_usd'] = df_['quantity'] * df_['price_usd']
    df_['value_btc'] = df_['value_usd'] / btc_price
    
    portfolio_usd = df_['value_usd'].sum()
    portfolio_btc = df_['value_usd'].sum() / btc_price
    
    print("Value of portfolio\n${:.2f} USD\n{:.4f} BTC".format(portfolio_usd, 
                                                               portfolio_btc))
    return df_p[cols]

In [60]:
get_portfolio(df, portfolio)

Value of portfolio
$431456.53 USD
65.1606 BTC


,name,symbol,price_usd,percent_change_24h,quantity,value_usd,value_btc
0,BTC Bitcoin,BTC,6621.43,6.52,2.045,13540.82435,2.045000
1,ETH Ethereum,ETH,315.05,3.29,20.170,6354.55850,0.959696
2,DASH Dash,DASH,411.15,7.93,1001.000,411561.15000,62.155932


## Conclusions <a id='next-steps'></a>
Having programatic access to ICO prices is a first step for many applications.<br>
Please use this information responsibly, IE [see the 2010 Flash Crash](https://www.theguardian.com/business/2015/apr/22/2010-flash-crash-new-york-stock-exchange-unfolded).

Here are some next steps:
- Do this for BTC units.
- Host the data on s3 to another open source outlet with programmatic access.
- Host a cloud instance that generates this dataset in a regular interval.
- Analysis of of BTC's price on alt-coins.

I think there is some good software to be written.<br>
This wass done as a passion project during a cold Novemeber weekend,<br>
if this was helpful to you, any suggestions, time, or donations are appreciated!

Wallet locations for donations:
<a id='TODO'>todo</a>